In [2]:
import pandas as pd
from gensim import utils
from gensim.models import FastText
from razdel import tokenize
from huggingface_hub import hf_hub_download
import fasttext
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

In [1]:
data = pd.read_csv("./train_dataset_Датасет/datasets/dataset_1.csv")

for i in range(1, 5):
    new_data = pd.read_csv(f"./train_dataset_Датасет/datasets/dataset_{i}.csv")
    data = pd.concat([data, new_data])

NameError: name 'pd' is not defined

In [4]:
model = FastText.load('fasttext.model')

In [37]:
class MyIter:
    def __iter__(self):
        for index, row in building.iterrows():
            tokens = list(tokenize(row["short_address"]))
            yield [_.text for _ in tokens]    

model = FastText(vector_size=128, window=5, min_count=1, workers=30, sg=1, negative=10)
model.build_vocab(corpus_iterable=MyIter())
total_examples = model.corpus_count
model.train(corpus_iterable=MyIter(), total_examples=len(building), epochs=20)

(8, 120)

In [3]:
building = pd.read_csv("/Users/kirill/Desktop/CP/digital-breakthrough/train_dataset_Датасет/additional_data/building_20230808.csv")
building_2 = building[["id", "full_address"]]
building = building[["id", "short_address"]]
building_1 = building.copy().sample(frac=1).reset_index(drop=True)
building_1 = building_1.rename(columns={"id": "id2", "short_address": "short_address2"})
building_zero = pd.concat([building, building_1], axis=1)
building_one = pd.concat([building, building.rename(columns={"id": "id2", "short_address": "short_address2"})], axis=1)
building_new = pd.concat([building_zero, building_one])
building_new = building_new.rename(columns={
    "id": "first_id",
    "short_address": "first_text",
    "id2": "second_id",
    "short_address2": "second_text"
})
building_new = building_new.sample(frac=1).reset_index(drop=True)

/var/folders/7b/_0j65qyd6_l801khvk_jh2yh0000gn/T/ipykernel_34999/1348858327.py:1: DtypeWarning: Columns (4,6,7,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  building = pd.read_csv("/Users/kirill/Desktop/CP/digital-breakthrough/train_dataset_Датасет/additional_data/building_20230808.csv")


In [4]:
building_new['label']= building_new['first_text'] == building_new['second_text']

In [5]:
building_new['label'] = building_new['label'].astype(np.int8)

In [6]:
building_new

,first_id,first_text,second_id,second_text,label
0,140015,"г.Зеленогорск, Церковная ул., д. 3/5, л. Б",196043,"г.Пушкин, сад-во ""Волхонское"", Добрососедская ...",0
1,65545,"г.Ломоносов, ул. Черникова, д. 34а, л. Б",102618,"пос. Лисий Нос, Глухариная ул., д. 3",0
2,58832,"г.Санкт-Петербург, Выборгское шоссе, д. 22, к....",212286,"г.Павловск, тер. СНТ Звёздочка, Рябиновая лини...",0
3,68525,"пос. Шушары, Н.Ижора, Кобринская ул., д. 41, л. А",68525,"пос. Шушары, Н.Ижора, Кобринская ул., д. 41, л. А",1
4,50360,"г.Сестрорецк, 37-й км Приморского шоссе, д. 1,...",5010,"г.Ломоносов, Полевая ул., д. 9, л. А",0
...,...,...,...,...,...
333285,2279,"г.Санкт-Петербург, ул. Десантников, д. 28, л. А",137734,"г.Красное Село, Можайский, Советская ул., д. 5...",0
333286,67263,"г.Красное Село, территория Дудергоф, Советская...",95431,"пос. Белоостров, Новое шоссе, д. 21, л. А",0
333287,204813,"г.Санкт-Петербург, Союзный пр., д. 4, л. А",204813,"г.Санкт-Петербург, Союзный пр., д. 4, л. А",1
333288,200958,"Комплекс защитных сооружений, л. АВ",66303,"пос. Серово, тер. СНТ ""Молодёжное"", д. 28, л. А",0


In [100]:
building = pd.read_csv("/Users/kirill/Desktop/CP/digital-breakthrough/train_dataset_Датасет/additional_data/building_20230808.csv")
building

/var/folders/7b/_0j65qyd6_l801khvk_jh2yh0000gn/T/ipykernel_30564/4252859258.py:1: DtypeWarning: Columns (4,6,7,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  building = pd.read_csv("/Users/kirill/Desktop/CP/digital-breakthrough/train_dataset_Датасет/additional_data/building_20230808.csv")


,id,prefix_id,district_id,house,corpus,liter,villa,parcel,full_address,is_updated,is_actual,type,municipality_id,short_address,post_prefix,build_number
0,56343,11132,35,12,NaN,А,NaN,NaN,"город Пушкин, Кедринская улица, дом 12",True,False,NaN,107.0,"г.Пушкин, Кедринская ул., д. 12",NaN,NaN
1,595,6987,38,4Б,NaN,NaN,NaN,NaN,"поселок Ушково, Пляжевая улица, дом 4Б",True,False,NaN,128.0,"пос. Ушково, Пляжевая ул., д. 4Б",NaN,NaN
2,7134,6469,15,30,2,Е,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова, до...",True,False,Нежилое,30.0,"г.Санкт-Петербург, пр. Маршала Жукова, д. 30, ...",198303,NaN
3,124415,7838,38,5,2,А,NaN,NaN,"поселок Белоостров, Дюны, Центральная улица, д...",True,False,Нежилое,110.0,"пос. Белоостров, Дюны, Центральная ул., д. 5, ...",NaN,NaN
4,185368,4224,38,28,NaN,Б,NaN,NaN,"поселок Песочный, Речная улица, дом 28, литера Б",True,False,NaN,118.0,"пос. Песочный, Речная ул., д. 28, л. Б",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166640,18941,5289,32,3,NaN,В,NaN,NaN,"г.Санкт-Петербург, Бородинская улица, дом 3, л...",True,True,Нежилое,4.0,"г.Санкт-Петербург, Бородинская ул., д. 3, л. В",190000.0,NaN
166641,18942,5031,35,34/3,NaN,А,NaN,NaN,"посёлок Александровская, 2-я линия, дом 34/3, ...",True,True,Жилое,109.0,"пос. Александровская, 2-я линия, д. 34/3, л. А",196631.0,NaN
166642,18943,5623,38,39,NaN,NaN,NaN,NaN,"посёлок Ушково, Детская улица, дом 39",True,False,NaN,128.0,"пос. Ушково, Детская ул., д. 39",NaN,NaN
166643,18944,5787,34,25,NaN,Б,NaN,NaN,"г.Санкт-Петербург, Заусадебная улица, дом 25, ...",True,False,Нежилое,68.0,"г.Санкт-Петербург, Заусадебная ул., д. 25, л. Б",NaN,NaN


In [7]:
building_2

,id,full_address
0,56343,"город Пушкин, Кедринская улица, дом 12"
1,595,"поселок Ушково, Пляжевая улица, дом 4Б"
2,7134,"г.Санкт-Петербург, проспект Маршала Жукова, до..."
3,124415,"поселок Белоостров, Дюны, Центральная улица, д..."
4,185368,"поселок Песочный, Речная улица, дом 28, литера Б"
...,...,...
166640,18941,"г.Санкт-Петербург, Бородинская улица, дом 3, л..."
166641,18942,"посёлок Александровская, 2-я линия, дом 34/3, ..."
166642,18943,"посёлок Ушково, Детская улица, дом 39"
166643,18944,"г.Санкт-Петербург, Заусадебная улица, дом 25, ..."


In [18]:
 building_new

,first_id,first_text,second_id,second_text,label
0,140015,"г.Зеленогорск, Церковная ул., д. 3/5, л. Б",196043,"г.Пушкин, сад-во ""Волхонское"", Добрососедская ...",0
1,65545,"г.Ломоносов, ул. Черникова, д. 34а, л. Б",102618,"пос. Лисий Нос, Глухариная ул., д. 3",0
2,58832,"г.Санкт-Петербург, Выборгское шоссе, д. 22, к....",212286,"г.Павловск, тер. СНТ Звёздочка, Рябиновая лини...",0
3,68525,"пос. Шушары, Н.Ижора, Кобринская ул., д. 41, л. А",68525,"пос. Шушары, Н.Ижора, Кобринская ул., д. 41, л. А",1
4,50360,"г.Сестрорецк, 37-й км Приморского шоссе, д. 1,...",5010,"г.Ломоносов, Полевая ул., д. 9, л. А",0
...,...,...,...,...,...
333285,2279,"г.Санкт-Петербург, ул. Десантников, д. 28, л. А",137734,"г.Красное Село, Можайский, Советская ул., д. 5...",0
333286,67263,"г.Красное Село, территория Дудергоф, Советская...",95431,"пос. Белоостров, Новое шоссе, д. 21, л. А",0
333287,204813,"г.Санкт-Петербург, Союзный пр., д. 4, л. А",204813,"г.Санкт-Петербург, Союзный пр., д. 4, л. А",1
333288,200958,"Комплекс защитных сооружений, л. АВ",66303,"пос. Серово, тер. СНТ ""Молодёжное"", д. 28, л. А",0


In [14]:
building_new['first_text'][0:len(building_2)] = np.where(building_new['first_id'][0:len(building_2)] == building_2['id'], building_2['full_address'], building_new['first_text'][0:len(building_2)]  )


/var/folders/7b/_0j65qyd6_l801khvk_jh2yh0000gn/T/ipykernel_34999/996016441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  building_new['first_text'][0:len(building_2)] = np.where(building_new['first_id'][0:len(building_2)] == building_2['id'], building_2['full_address'], building_new['first_text'][0:len(building_2)]  )


In [15]:
building_new['first_text'][len(building_2):2*len(building_2)] = np.where(building_new['first_id'][len(building_2):2*len(building_2)].reset_index(drop=True)  == building_2['id'].reset_index(drop=True) , building_2['full_address'], building_new['first_text'][len(building_2):2*len(building_2)] )

/var/folders/7b/_0j65qyd6_l801khvk_jh2yh0000gn/T/ipykernel_34999/912086571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  building_new['first_text'][len(building_2):2*len(building_2)] = np.where(building_new['first_id'][len(building_2):2*len(building_2)].reset_index(drop=True)  == building_2['id'].reset_index(drop=True) , building_2['full_address'], building_new['first_text'][len(building_2):2*len(building_2)] )


In [16]:
building_new

,first_id,first_text,second_id,second_text,label
0,140015,"г.Зеленогорск, Церковная ул., д. 3/5, л. Б",196043,"г.Пушкин, сад-во ""Волхонское"", Добрососедская ...",0
1,65545,"г.Ломоносов, ул. Черникова, д. 34а, л. Б",102618,"пос. Лисий Нос, Глухариная ул., д. 3",0
2,58832,"г.Санкт-Петербург, Выборгское шоссе, д. 22, к....",212286,"г.Павловск, тер. СНТ Звёздочка, Рябиновая лини...",0
3,68525,"пос. Шушары, Н.Ижора, Кобринская ул., д. 41, л. А",68525,"пос. Шушары, Н.Ижора, Кобринская ул., д. 41, л. А",1
4,50360,"г.Сестрорецк, 37-й км Приморского шоссе, д. 1,...",5010,"г.Ломоносов, Полевая ул., д. 9, л. А",0
...,...,...,...,...,...
333285,2279,"г.Санкт-Петербург, ул. Десантников, д. 28, л. А",137734,"г.Красное Село, Можайский, Советская ул., д. 5...",0
333286,67263,"г.Красное Село, территория Дудергоф, Советская...",95431,"пос. Белоостров, Новое шоссе, д. 21, л. А",0
333287,204813,"г.Санкт-Петербург, Союзный пр., д. 4, л. А",204813,"г.Санкт-Петербург, Союзный пр., д. 4, л. А",1
333288,200958,"Комплекс защитных сооружений, л. АВ",66303,"пос. Серово, тер. СНТ ""Молодёжное"", д. 28, л. А",0


In [5]:
model = FastText.load('fasttext.model')
vector = np.array(model.wv.vectors_vocab)

In [6]:
keys = model.wv.key_to_index
a = np.array(list(keys.keys()))

In [7]:
a = a.reshape(-1,1)

In [144]:
data = np.concatenate((a, vector), axis = 1)


In [9]:
building = pd.read_csv('./data/train.csv')
building

,address1,address2,label
0,kaka,pisy,1
1,хуй,Жопа,0
2,хуй,уйх,1


In [10]:
b1 = list(building['address1'])
b2 = list(building['address2'])
vec1 = np.zeros((len(b1), 128))
vec2 = np.zeros((len(b2), 128))

In [12]:
for i in range(len(b1)):
    vec1[i] = model.wv.get_vector(b1[i])
    vec2[i] = model.wv.get_vector(b2[i])

In [13]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

In [14]:
class CustomTextDataset(Dataset):
    def __init__(self, x, y, labels):
        self.labels = labels
        self.x= x
        self.y = y
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        label = self.labels[idx]
        address1 = self.x[idx]
        address2 = self.y[idx]
        sample = {"address1": address1, "address2": address2, "label": label}
        return sample

In [15]:
vec1, vec2, labels = torch.tensor(vec1), torch.tensor(vec2), torch.tensor(building['label'])

In [16]:
labels.shape

torch.Size([3])

In [17]:
d = CustomTextDataset(vec1, vec2, building['label'])

In [23]:
for i  in range(len(d)):
    # Print the 'text' data of the batch
    print('x: ', d[i]['address1'].shape)
    # Print the 'class' data of batch
    print('y: ', d[i]['address2'].shape, '\n')
    print('Class data: ', d[i]['label'], '\n')

x:  torch.Size([128])
y:  torch.Size([128]) 

Class data:  1 

x:  torch.Size([128])
y:  torch.Size([128]) 

Class data:  0 

x:  torch.Size([128])
y:  torch.Size([128]) 

Class data:  1 



In [107]:
df.to_pickle("my_data.bin")

In [4]:
model =  FastText(vector_size=128, window=5, min_count=1, workers=30, sg=1, negative=10)

In [6]:
model = FastText.load('fasttext.model')

In [7]:
vector_index = []

for index, row in tqdm(building.iterrows()):
    tokens = list(tokenize(row["short_address"]))
    tokens = [_.text for _ in tokens]
    predict = np.array([model.wv[token] for token in tokens])
    predict = np.mean(predict, axis=0)
    predict = predict / np.linalg.norm(predict)
    vector_index.append((row['id'], predict))

0it [00:00, ?it/s]

166645it [00:47, 3496.10it/s]


In [8]:
def get_id(text):
    vector = model.wv[text] 
    tokens = list(tokenize(text))
    tokens = [_.text for _ in tokens]
    vector = np.array([model.wv[token] for token in tokens])
    vector = np.mean(vector, axis=0)
    vector = vector / np.linalg.norm(vector).reshape(1, -1) 
    max_sym = -1
    best_id = 0
    step = 10000

    for ind in range(0, len(vector_index), step):
        vectors = np.array([i[1] for i in vector_index[ind: ind + step]]).T
        syms = vector @ vectors
        best_local_ind = np.argmax(syms)
        sym = syms[0][best_local_ind]
        if sym > max_sym:
            max_sym = sym
            best_id = vector_index[ind + best_local_ind][0]
    return best_id, max_sym

In [9]:
cnt = 0
ind = 0
for index, row in tqdm(data.iterrows()):
    ind += 1
    best_id, max_sym = get_id(row["address"])
    #print(best_id, row['target_building_id'])
    if best_id == row['target_building_id']:
        cnt += 1
    if ind == 1000:
        break
print(cnt / 1000)

999it [02:18,  7.21it/s]

0.396


In [191]:
model.save("fasttext.model")